In [1]:
!pip install medmnist

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://pypi.ngc.nvidia.com


In [2]:
from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

import medmnist
from medmnist import INFO, Evaluator

In [3]:
print(f"MedMNIST v{medmnist.__version__} @ {medmnist.HOMEPAGE}")

MedMNIST v3.0.1 @ https://github.com/MedMNIST/MedMNIST/


In [4]:
data_flag = 'breastmnist'
# data_flag = 'breastmnist'
download = True

NUM_EPOCHS = 3
BATCH_SIZE = 128
lr = 0.001

info = INFO[data_flag]
task = info['task']
n_channels = info['n_channels']
n_classes = len(info['label'])

DataClass = getattr(medmnist, info['python_class'])

In [5]:
# preprocessing
data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])

# load the data
train_dataset = DataClass(split='train', transform=data_transform, download=download)
test_dataset = DataClass(split='test', transform=data_transform, download=download)

pil_dataset = DataClass(split='train', download=download)

# encapsulate data into dataloader form
train_loader = data.DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True)
train_loader_at_eval = data.DataLoader(dataset=train_dataset, batch_size=2*BATCH_SIZE, shuffle=False)
test_loader = data.DataLoader(dataset=test_dataset, batch_size=2*BATCH_SIZE, shuffle=False)

Using downloaded and verified file: /Users/yuqi/.medmnist/breastmnist.npz
Using downloaded and verified file: /Users/yuqi/.medmnist/breastmnist.npz
Using downloaded and verified file: /Users/yuqi/.medmnist/breastmnist.npz


In [6]:
print(train_dataset)
print("===================")
print(test_dataset)

Dataset BreastMNIST of size 28 (breastmnist)
    Number of datapoints: 546
    Root location: /Users/yuqi/.medmnist
    Split: train
    Task: binary-class
    Number of channels: 1
    Meaning of labels: {'0': 'malignant', '1': 'normal, benign'}
    Number of samples: {'train': 546, 'val': 78, 'test': 156}
    Description: The BreastMNIST is based on a dataset of 780 breast ultrasound images. It is categorized into 3 classes: normal, benign, and malignant. As we use low-resolution images, we simplify the task into binary classification by combining normal and benign as positive and classifying them against malignant as negative. We split the source dataset with a ratio of 7:1:2 into training, validation and test set. The source images of 1×500×500 are resized into 1×28×28.
    License: CC BY 4.0
Dataset BreastMNIST of size 28 (breastmnist)
    Number of datapoints: 156
    Root location: /Users/yuqi/.medmnist
    Split: test
    Task: binary-class
    Number of channels: 1
    Meaning

## Data Preprocessing

# We first compare their singular values

## We first use standard SVD to check their singular values

In [7]:
from SVD import standard_svd

In [9]:
type(x)

torch.Tensor

In [8]:
from SVD import standard_svd
svd_standard={}
singular_values={}
x_train={}
y_train={}
for i in range(10):
    x_train[str(i)],y_train[str(i)]=train_dataset[i]
    svd_standard[str(i)],singular_values[str(i)]=standard_svd(x_train[str(i)][0])

## then we try our first sketching matrix

In [11]:
!pip install utils

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple, https://pypi.ngc.nvidia.com


## Uniform Sketching Matrices

In [ ]:
uniform_matrices={}
from sketching import uniform_sketching_matrix
from QB_decomposition_fixed_precision import randQB_FP_auto
for i in range (10):
    uniform_matrices[str(i)]=uniform_sketching_matrix(28, len(singular_values[str(i)])+5)
#get their QB decomposition
#rank_approximated={}
Q_uniform={}
B_uniform={}
iteration_uniform={}
error_uniform={}
for i in range(10):
    Q,B,error,iteration=randQB_FP_auto(x_train[str(i)][0],0.01,4,4,uniform_matrices[str(i)])
    Q_uniform[str(i)]=Q
    B_uniform[str(i)]=B
    error_uniform[str(i)]=error
    iteration_uniform[str(i)]=iteration

88.42498808427192
fail to converge
246.6902514377964
fail to converge
86.56522173900105
fail to converge
108.26216001705322
fail to converge


IndexError: index 3 is out of bounds for dimension 0 with size 3

In [12]:
#then we do SVM based on the new matrix
rand_singular_values={}
for i in range(10):
    _, rand_singular_values[str(i)]=standard_svd(B)

In [13]:
#compare singular values
import numpy as np
from scipy.linalg import svd
from numpy.linalg import qr

def randomized_svd(A, k, p=5, n_iter=2):
    """
    Perform Randomized SVD on matrix A.
    
    Parameters:
        A: ndarray, shape (m, n)
           Input matrix to decompose.
        k: int
           Target rank (number of singular values/vectors to compute).
        p: int, optional (default=5)
           Oversampling parameter to improve approximation accuracy.
        n_iter: int, optional (default=2)
           Number of iterations for power iteration (improves accuracy).
           
    Returns:
        U: ndarray, shape (m, k)
           Approximated left singular vectors.
        S: ndarray, shape (k,)
           Approximated singular values.
        V: ndarray, shape (k, n)
           Approximated right singular vectors.
    """
    # Step 1: Random sampling
    m, n = A.shape
    random_matrix = np.random.randn(n, k + p)
    Y = A @ random_matrix

    # Step 2: Power iteration (optional, improves accuracy for ill-conditioned matrices)
    for _ in range(n_iter):
        Y = A @ (A.T @ Y)

    # Step 3: QR decomposition
    Q, _ = qr(Y, mode='reduced')

    # Step 4: Project matrix to lower-dimensional space
    B = Q.T @ A

    # Step 5: Compute SVD on the smaller matrix
    U_tilde, S, Vt = svd(B, full_matrices=False)

    # Step 6: Reconstruct the left singular vectors
    U = Q @ U_tilde

    return U[:, :k], S[:k], Vt[:k, :]


In [15]:
print(singular_values[str(0)])

[9.28383064e+00 2.60005903e+00 2.41792083e+00 1.46680355e+00
 1.12029254e+00 1.05091131e+00 8.14428389e-01 6.26211822e-01
 5.34207582e-01 4.22860980e-01 3.79819393e-01 3.05890709e-01
 2.81926990e-01 2.17790991e-01 1.92904517e-01 1.50928587e-01
 1.34227872e-01 1.23457581e-01 1.04857974e-01 9.30917487e-02
 7.62561411e-02 6.59982264e-02 5.83461151e-02 5.18235788e-02
 2.83046421e-02 1.98337790e-02 8.84390064e-03 1.74836244e-03]


In [ ]:
result=randomized_svd(np.array(x_train['0'][0].type(torch.DoubleTensor)),20)
#These are some trials

(array([[-0.13484274, -0.0015452 , -0.12138627,  0.00567624, -0.06577583,
          0.18606355, -0.05785038,  0.00839666,  0.00222614,  0.13494908,
         -0.18826216,  0.11074406,  0.11898215, -0.17174566,  0.00941675,
         -0.18637832, -0.34427258, -0.32151842,  0.23231473,  0.24658771],
        [ 0.04528188,  0.1376931 , -0.18435122, -0.26042993, -0.07560154,
          0.25522462, -0.06984181,  0.22471956, -0.09997705, -0.19194721,
         -0.03127987,  0.20423388, -0.45357173,  0.02641731, -0.07095654,
          0.24463966, -0.19004559,  0.11124424, -0.25393799, -0.05798011],
        [ 0.08961659,  0.27314251, -0.02964677, -0.1177919 , -0.09609545,
         -0.00255461, -0.34605058, -0.03858863, -0.1454312 ,  0.31093664,
          0.06348096,  0.19982519, -0.06532405,  0.3762924 ,  0.11205452,
         -0.38223869, -0.2216049 ,  0.08696676, -0.10630653,  0.11270327],
        [ 0.10686712,  0.31119042, -0.04016767,  0.08066759,  0.30323564,
          0.01899802, -0.12247435, 

## Gaussian Sketching matrices

In [1]:
gaussian_matrices={}
from sketching import gaussian_sketching_matrix
from QB_decomposition_fixed_precision import randQB_FP_auto
for i in range (10):
    gaussian_matrices[str(i)]=gaussian_sketching_matrix(28, len(singular_values[str(i)])+5)
#get their QB decomposition
#rank_approximated={}
Q_gaussian={}
B_gaussian={}
error_gaussion={}
iteration_gaussian={}
#print(gaussian_matrices[str(0)])
for i in range(10):
    Q,B,error,iteration=randQB_FP_auto(train_dataset[i][0][0],0.01,4,4,gaussian_matrices[str(i)])
    #rank_approximated[str(i)]=k
    Q_gaussian[str(i)]=Q
    B_gaussian[str(i)]=B
    error_gaussion[(str(i))]=error
    iteration_gaussian[str(i)]=iteration

    

NameError: name 'singular_values' is not defined

## Radamecher Sketching Matrices

## Johnson-Linderstrauss Transform Code

In [ ]:
from jlt.linearMapping import calculate_R
from QB_decomposition_fixed_precision import randQB_FP_auto
for i in range(10):
    JLT_matrix=calculate_R(28,k=None,1,random_seed=21,swr=True)
    

## Clarkson_Woodff

In [ ]:
from sketching import cwt_sketch_matrix
